## RESOURCES
**2019-03-12 18:43:07**

#### PACKAGES

In [258]:
# DATA
import pandas as pd
import numpy as np

# READING/WRITING
import json
import csv

# PARSING
from bs4 import BeautifulSoup, NavigableString, Tag
import lxml.html
import requests
from string import capwords
from titlecase import titlecase

# DATES/TIMES
from datetime import datetime
import pytz
from pytz import timezone
import time

# SYSTEM
import os, sys
from pathlib2 import Path
import ConfigParser

# MULTIPROCESSING
import multiprocessing
import multiprocessing_logging
import parmap
import tqdm

# ERROR HANDLING
import errno
import traceback

# SINGLE PROCESS LOGGING
import logging
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# MISC
import collections
import re
import copy
import random
from pprint import pprint
import uuid
import time
import os
import shutil
from pathlib2 import Path

import sys
import threading
import traceback


# print 'ALL MODULES LOADED'

# # FOR TESTING INDIVIDUAL FUNCTIONS
# test_path = '../data/samples/'
# test_fid = '110/'
# test_file = 'fid_110_tid_706047_pg_1.html'

# html_page = load_html_file(test_path + test_fid + test_file)

## FUNCTIONS

### POSTS

#### GET HTML POSTS

In [259]:
def get_html_posts(html_page):
    
    post_tables = html_page.find_all('table', {'id': re.compile('post')})
    post_rows = [post_table.find_all('tr', recursive=False) for post_table in post_tables]
    
    return post_rows
    
# html_posts = get_html_posts(html_page)
# len(html_posts)

#### GET POST DATE AND TIME

In [260]:
def get_date_time(el):
    
    top_bar = el.find_all('td', {'class': 'thead'})[0]
    top_bar_text = top_bar.get_text().encode('ascii', 'ignore').decode('ascii')
    top_bar_text = top_bar_text.replace('#', '').replace(',', '')
    
    if ('Today' or 'Yesterday') in top_bar_text:
        
        date_time = top_bar_text
            
    else:

        local_datetime = datetime.strptime(top_bar_text.split(' ', 1)[1], '%m-%d-%Y %I:%M %p')
        date_time = convert_to_utc(local_datetime, 'US/Mountain')
    
#     return utc_datetime
    return date_time

# get_date_time(html_posts[0][0]) # '2005-01-07 11:17:00 UTC+0000'

#### GET POST ID

In [261]:
def get_post_id(el):
    
    post_id_link = el.find_all('a', {'id': re.compile('postcount')})
    pid = post_id_link[0]['id'].split('t')[-1]
    
    return pid

# get_post_id(html_posts[0][0])

#### GET POST NUM

In [262]:
def get_post_num(el):
    
    top_bar = el.find_all('td', {'class': 'thead'})[0]
    top_bar_text = top_bar.get_text().encode('ascii', 'ignore').decode('ascii')
    top_bar_text = top_bar_text.replace('#', '').replace(',', '')
    
    post_count = top_bar_text.split(' ', 1)[0]
    
    return post_count

# get_post_num(html_posts[0][0]) #41

#### GET POST TEXT

In [263]:
def get_post_text(el):
    
    post_msg_el = el.find_all('div', {'id': re.compile('post_message')})[0]
    
    try: 
        post_msg_el.div.decompose() # TO REMOVE QUOTE

    except AttributeError:

        pass # LET'S EVENTUALLY LOG SOMETHING HERE

    msg_text = post_msg_el.get_text(" ", strip=True)
    
    return msg_text

# reg_trs = posts[0].find_all('tr', recursive=False)
# quote_trs = posts[8].find_all('tr', recursive=False)
# print 'NO QUOTE', '\n', get_post_message(reg_trs[2]), '\n' # "Met a boy, because if he...with a womans heart."
# print 'WITH QUOTE', '\n', get_post_message(quote_trs[2]) # "I am glad this information...which must be horrible"
# get_post_text(html_posts[0][2])

#### GET FORUM INFO

In [264]:
def get_forum_id(file_path):
    
    file_name = file_path.split('/')[-1]
    
    return file_name.split('_')[1]

def get_thread_id(file_path):
    
    file_name = file_path.split('/')[-1]
    
    return file_name.split('_')[3]

def get_page_number(file_path):
    
    file_name = file_path.split('/')[-1]
    
    return file_name.split('_')[-1].split('.')[0]

# file_path = '../examples/threads/parsing/1506/fid_1506_tid_97875_pg_4.html'
# print 'FORUM:', get_forum_id(file_path), 'THREAD ID:', get_thread_id(file_path), 'PAGE:', get_page_number(file_path)

#### GET POST INFO

In [265]:
def get_post_info(file_name, post_el):
    
    post = collections.OrderedDict()
    
    post['date'] = get_date_time(post_el[0])
    
    post['fid'] = get_forum_id(file_name)
    post['tid'] = get_thread_id(file_name)
    post['pg'] = get_page_number(file_name)
    
    post['pid'] = get_post_id(post_el[0])
    post['post_num'] = get_post_num(post_el[0])
    
    post['user'] = get_user_name(post_el[1])
    post['uid'] = get_user_id(post_el[1])

    post['text'] = get_post_text(post_el[2])
    
    return post

# thread_path = '../examples/threads/parsing/'
# forum_id = '39/'
# html_file = 'fid_39_tid_527690_pg_1.html'
# html_page = load_html_file(thread_path + forum_id + html_file)


# dir_path = '../examples/threads/parsing/39/'
# file_name = 'fid_39_tid_527690_pg_1.html'
# html_page = load_html_file(dir_path + file_name)
# html_posts = get_html_posts(html_page)

# post = get_post_info(file_name, html_posts[0])
# posts = [get_post_info(html_file_path, html_post) for html_post in html_posts]

# for key, val in post.items():
#     print key + ':', val

#### CONVERT TO UTC

In [266]:
def convert_to_utc(local_time, local_tz, string=True):
    
    fmt = '%Y-%m-%d %H:%M:%S %Z%z'
    utc = pytz.utc
    local_timezone = timezone(local_tz)
    
    local_datetime = local_timezone.localize(local_time)
    utc_datetime = local_datetime.astimezone(utc)
    
    if string:
        utc_datetime = utc_datetime.strftime(fmt)
    
    return utc_datetime

# local_datetime = datetime.strptime('02-17-2018 07:50 AM', '%m-%d-%Y %I:%M %p')
# convert_to_utc(local_datetime, 'US/Mountain') # '2018-02-17 14:50:00 UTC+0000'

### USER

#### GET USERNAME

In [267]:
def get_user_name(el):

    user_el = el.find_all('a', {'class': 'bigusername'})[0]
    user_name = user_el.get_text()
    
    return user_name

# user_name = get_user_name(trs[1])
# print user_name # maytayah

#### GET USER ID

In [268]:
def get_user_id(el):

    user_el = el.find_all('a', {'class': 'bigusername'})[0]
    user_id = user_el['href'].split('=')[-1]
    
    return user_id

# user_id = get_user_id(trs[1])
# print user_id # 380798

#### GET USER ROLE

In [269]:
def get_user_role(el):
    
    user_roles = {'red': 'admin', 'blue': 'super', 'purple': 'site', 'darkgreen': 'mod', 
                  'magenta': 'leave', 'MediumTurquoise': 'card', 'black': 'user'}
    
    try:
        user_el = el.find_all('a', {'class': 'bigusername'})[0]
        # <a class="bigusername" href="member.php?u=380798"><font color="purple"><b>maytayah</b></font></a>
        color = user_el.font['color']
    
    # if user is simply 'registered' then it won't have <font> element
    except TypeError as err:
        
        color = u'black'
        
    return user_roles[color]

# # Administrator: red
# # PTO Super Moderator: blue
# # PTO Site Moderator: purple
# # PTO Moderator: darkgreen
# # Moderator On Leave: magenta
# # PTO Card Swap Host: MediumTurquoise
# # User: black

# reg_trs = posts[0].find_all('tr', recursive=False)
# mod_trs = posts[6].find_all('tr', recursive=False)

# print 'USER ONE:', get_user_role(reg_trs[1]), '\n', 'USER TWO:', get_user_role(mod_trs[1])

#### GET USER ROLE DESCRIPTION

In [270]:
def get_user_role_desc(el):
    
    return el.find_all('div', {'class': 'smallfont'})[0].text

# # WILL INCLUDE BANNED AND ACCOUNT CLOSED
# reg_trs = posts[0].find_all('tr', recursive=False)
# mod_trs = posts[6].find_all('tr', recursive=False)

# print 'USER ONE:', get_user_role_desc(reg_trs[1]), '\n', 'USER TWO:', get_user_role_desc(mod_trs[1])

#### GET JOIN DATE

In [271]:
def get_join_date(el):
    
    text = el.find(text=re.compile('Join'))
    
    if text: 
        join_date = text.split(':')[1].strip()
    else:
        join_date = None
    
    return join_date
    
# print 'JOIN:', get_join_date(html_posts[30][1])

#### GET LOCATION

In [272]:
def get_location(el):
    
    text = el.find(text=re.compile('Location'))
    
    if text:
        location = text.split(':')[1].strip()
    else:
        location = None
    
    return location

# print 'LOCATION:', get_location(html_posts[30][1])

#### GET NUM POSTS

In [273]:
def get_num_posts(el):
    
    text = el.find(text=re.compile('Posts'))
    
    if text:
        num_posts = text.split(':')[1].strip()
    else:
        num_posts = None
    
    return num_posts

# print 'POSTS:', get_num_posts(html_posts[30][1])

#### GET NUM THANKS

In [274]:
def get_num_thanks(el):
    
    text = el.find(text=re.compile('Thanks'))
    
    if text:
        num_thanks = text.split(':')[1].strip()
    else:
        num_thanks = None
        
    return num_thanks

# print 'THANKS:', get_num_thanks(html_posts[30][1])

#### GET NUM THANKED

In [275]:
def get_num_thanked(el):
        
    text = el.find(text=re.compile('Thanked'))
    
    if text:
        num_thanked = text.split(' ')[1].strip()
    else:
        num_thanked = None
    
    return num_thanked

def get_num_posts_thanked(el):
    
    text = el.find(text=re.compile('Thanked'))
    
    if text:
        num_posts_thanked = text.split(' ')[4].strip()
    else:
        num_posts_thanked = None
    
    return num_posts_thanked

# print 'THANKED', get_num_thanked(html_posts[30][1]), 'TIMES IN', get_num_posts_thanked(html_posts[30][1]), 'POSTS'

#### GET USER INFO

In [276]:
def get_user_info(el):
    
    user = collections.OrderedDict()
    
    user['user'] = get_user_name(el[1])
    user['uid'] = get_user_id(el[1])
    user['role'] = get_user_role(el[1])
    user['role_desc'] = get_user_role_desc(el[1])
    user['join_date'] = get_join_date(el[1])
    user['location'] = get_location(el[1])
    user['posts'] = get_num_posts(el[1])
    user['thanks'] = get_num_thanks(el[1])
    user['thanked'] = get_num_thanked(el[1])
    user['posts_thanked'] = get_num_posts_thanked(el[1])
    
    return user

In [277]:
# html_page = load_html_file(dir_path + '/39/fid_39_tid_81190_pg_10.html')
# html_posts = get_html_posts(html_page)
# post_nums = [29, 30, 31]

# for post_num in post_nums:    
    
#     user = get_user_info(html_posts[post_num])
    
#     for key, val in user.items():
#         print key + ':', val
#     print ''

### THANK YOU'S

#### GET HTML THANK YOUS

In [278]:
def get_html_thank_yous(html_page):
        
    return html_page.find_all(id=re.compile('post_thanks_box'))
    
# thanks_none = get_html_thank_yous(html_file_none)
# thanks_many = get_html_thank_yous(html_file_many)
# len(thanks_none), len(thanks_many)
# thanks_yous = get_html_thank_yous(html_page)
# len(thanks_yous)

#### GET THANKS LINKS

In [279]:
def get_thanks_links(el):
    
    return el.find_all('a')

# no_links = [get_thanks_links(thanks) for thanks in thanks_none]
# links = [get_thanks_links(thanks) for thanks in thanks_many]

# len(no_links), len(links)
links = get_thanks_links

#### GET THANKED POST ID

In [280]:
def get_thanked_post_id(el):
    
    return el['id'].split('_')[-1]

# pid_none = get_thanked_post_id(thanks_none[0])
# pid_many = get_thanked_post_id(thanks_many[0])
# pid_none, pid_many

#### GET DATE OF THANKS

In [281]:
def get_date_of_thanks(el):
        # <a>...</a>(02-27-2015),</div>        
    ascii_date = el.next_sibling
    # u'\xa0(05-13-2009),
    decoded_date = ascii_date.encode('ascii', 'ignore').decode('ascii')
    # (05-13-2009),
    date_of_thanks = re.sub('[(),]', '', decoded_date)
    
    return date_of_thanks

# get_date_of_thanks(links[0])

#### GET THANKER USER ID

In [282]:
def get_thanker_user_id(el):
    
    return el['href'].split('=')[-1]

# get_thanker_user_id(links[0])

#### GET THANKER USER NAME

In [283]:
def get_thanker_user_name(el):
    
    return el.text

# get_thanker_user_name(links[0])

#### GET THANK YOU INFO

In [284]:
def get_thank_you_info(thanks_el, link_el):
    
    if link_el:
    
        thank_you = collections.OrderedDict()

        thank_you['pid'] = get_thanked_post_id(thanks_el)
        thank_you['date'] = get_date_of_thanks(link_el)
        thank_you['from_uid'] = get_thanker_user_id(link_el)
        thank_you['from_user'] = get_thanker_user_name(link_el)

        return thank_you

### FILE HANDLING

#### GET FORUM DIRECTORIES

In [285]:
def get_forum_directories(dir_path, skip=[]):
    
    directories = [str(path) for path in Path(dir_path).iterdir() 
                   if (path.is_dir() and path.stem not in skip)]
    
    return directories

# skip = ['logs', 'csv', 'error']
# get_forum_directories('../examples/threads/parsing/', skip)

In [286]:
def get_forum_files(forum_path):
    
    thread_path = Path(forum_path)
    all_files = [str(html_file) for html_file in thread_path.rglob('*.html')]
    
    return all_files


# test_path = '../examples/threads/parsing/1506'
# get_forum_files(test_path)

#### GET FORUM FILE DICTS

In [287]:
def get_forum_file_dicts(forum_path, list_size):
    
    all_files = get_forum_files(forum_path)
    
    chunks = split_file_list(all_files, 3)
    
    forum_id = forum_path.split('/')[4]
    
    forum_file_dicts = [{'fid': forum_id, 'files': chunk} for chunk in chunks]

    return forum_file_dicts

# htmlpath = '../examples/threads/parsing/1506'
# size = 3
# forum_file_dicts = get_forum_file_dicts(htmlpath, size)
# for forum_dict in forum_file_dicts:
#     print forum_dict['fid'] + ':', len(forum_dict['files'])

#### GET FORUM FILE DICT

In [288]:
def get_forum_file_dict(dir_path):
    
    thread_path = Path(dir_path)
    forum_files = {'fid': thread_path.stem, 'files': [str(html_file) for html_file in thread_path.rglob('*.html')]}
    
    return forum_files

#### LOAD HTML FILE

In [289]:
def load_html_file(file_path):
    
    html_page = open(file_path, 'r').read()
    
    posts_page = str(BeautifulSoup(html_page, 'html.parser'))
    no_literals = posts_page.replace('\n', '').replace('\t', '').replace('\r', '')
    
    return BeautifulSoup(no_literals, 'html.parser') 
    
# thread_path = '../examples/threads/parsing/'
# forum_id = '39/'
# html_file = 'fid_39_tid_527690_pg_1.html'
# html_page = load_html_file(thread_path + forum_id + html_file)

#### CREATE LOGGER

In [290]:
def create_logger(file_path):

    log_file = 'PT_' + datetime.strftime(datetime.today(), '%d_%m_%Y_%I_%M_%p') + '.log'
    msgfmt = '%(asctime)s %(levelname)s: %(message)s'
    datefmt= '%d/%m/%Y %I:%M:%S %p'
    
    reload(logging)
    logging.basicConfig(filename=file_path + '/' + log_file, level=logging.INFO, format=msgfmt, datefmt=datefmt)
    logging.Formatter.converter = time.gmtime # CHANGE TO UTC TIME
    logger = logging.getLogger()
    
    return logger

# logger = create_forum_logger('../examples/threads/parsing/logs')
# html_file = '../examples/threads/parsing/39/fid_39_tid_81190_pg_10.html'.split('/')[-1]

# try:
#     1/0
# except Exception as err:

#     logger.error(html_file + ' - ' + err.__class__.__name__ + ': ' + str(err))
#     print html_file + ' - ' + err.__class__.__name__ + ': ' + str(err)
# # 25/08/2018 01:53:50 AM ERROR: fid_39_tid_81190_pg_10.html - ZeroDivisionError: integer division or modulo by zero

#### CREATE MULTIPROCESSING LOGGER

In [291]:
def create_multiprocessing_logger(log_path, info_fname, error_fname='error.log', stream=True):

    msg_fmt = '%(name)s %(asctime)s %(levelname)s: %(message)s'
    date_fmt= '%d/%m/%Y %I:%M:%S %p'

    logging.basicConfig(level=logging.INFO, format=msg_fmt, datefmt=date_fmt)
    logging.Formatter.converter = time.gmtime # CHANGE TO UTC TIME

    file_handler = logging.FileHandler(log_path + info_fname)
    file_handler_formatter = logging.Formatter(fmt=msg_fmt, datefmt=date_fmt)
    file_handler.setFormatter(file_handler_formatter)
    file_handler.setLevel(logging.INFO)

    error_handler = logging.FileHandler(log_path + error_fname)
    error_handler_formatter = logging.Formatter(fmt=msg_fmt, datefmt=date_fmt)
    error_handler.setFormatter(error_handler_formatter)
    error_handler.setLevel(logging.ERROR)
    
    logger = logging.getLogger()
    logger.handlers = []
    logger.addHandler(error_handler)
    logger.addHandler(file_handler)
    
    if stream:
        stream_handler = logging.StreamHandler()
        stream_handler_formatter = logging.Formatter(fmt=msg_fmt, datefmt=date_fmt)
        stream_handler.setFormatter(stream_handler_formatter)
        stream_handler.setLevel(logging.INFO)
        logger.addHandler(stream_handler)
        
    return logger

#### PRINT FINAL RESULTS

In [292]:
def print_final_results(thread_dfs, thread_data, headers, rows=5):
    
    for thread_df, data, header in zip(thread_dfs, thread_data, headers):
        print 'ROWS:', len(data), 'COLS:', len(header['headers']), 'SHAPE:', thread_df.shape
    print ''
    
    if rows:

        print thread_dfs[0][['fid', 'tid', 'pg', 'pid']].head(rows)
    #     print thread_dfs[0][['fid', 'tid', 'pg', 'pid']].tail(rows)

        print ''
        print thread_dfs[1][['user','join_date', 'location', 'posts']].head(rows)
    #     print thread_dfs[1][['user','join_date', 'location', 'posts']].tail(rows)

        print ''
        print thread_dfs[2][['date', 'pid', 'from_uid', 'from_user']].head(rows)
    #     print thread_dfs[2][['date', 'pid', 'from_uid', 'from_user']].tail(rows)
        print ''

#### CREATE CSV PAGE FILE PATH

In [293]:
def create_csv_page_file_path(html_file_path, out_path, data_type):
    # '../examples/threads/parsing/1506/ --> fid_1506_tid_97875_pg_4 - .html' + '_type.csv = fid_1506_tid_97875_pg_4_posts.csv
    file_name = html_file_path.split('/')[-1][:-5] + ('_' + data_type + '.csv')
    csv_file_path = out_path + 'pages/' + data_type + '/' + file_name
    
    return csv_file_path

# hfp = '../data/forums/1506/fid_1506_tid_97875_pg_4.html'
# op = '../data/csv/'

# hfp = '../examples/threads/parsing/forums/39/fid_39_tid_81190_pg_2.html'
# op = '../examples/threads/parsing/csv/'
# create_csv_page_file_path(hfp, op, 'posts') # '../examples/threads/parsing/csv/pages/posts/fid_39_tid_81190_pg_2_posts.csv'

#### CREATE CSV FORUM FILE PATH

In [294]:
def create_csv_forum_file_path(csv_path, fid, data_type):
    
    csv_file = 'fid_' + fid + '_' + data_type + '.csv'
    
    return str(Path(csv_path).joinpath(data_type, csv_file))

# create_csv_forum_file_path(csv_path, '48', 'posts') # '../data/csv/forums/posts/fid_48_posts.csv'

#### WRITE PAGE DATA TO CSV

In [295]:
# MULTIPROCESSING VERSION
def write_page_data_to_csv(thread_data, html_file_path, out_path):
    
    posts_headers = {'type': 'posts', 'headers': ['date', 'fid', 'tid', 'pg', 'post_num', 'pid', 'user', 'uid', 'text']}
    users_headers = {'type': 'users', 'headers': ['user', 'uid', 'role', 'role_desc', 'join_date', 'location', 'posts', 'thanks', 'posts_thanked']}
    thanks_headers = {'type': 'thanks', 'headers': ['date', 'pid', 'from_uid', 'from_user']}

    headers = [posts_headers, users_headers, thanks_headers]
    
    for data, header in zip(thread_data, headers):

        csv_file_path = create_csv_file_path(html_file_path, out_path, header['type'])

        df = pd.DataFrame(data, columns=header['headers'])
        df.to_csv(csv_file_path, index=False, encoding='utf-8')
                    
#         print file_path, df.shape

# TODO: MAKE TESTS THAT DON'T RELY ON PREVIOUSLY RUN CODE

# thread_data = [posts, users, thank_yous]
# html_file_path = '../examples/threads/parsing/csv/pages/posts/fid_1506_tid_97875_pg_4_posts.csv'
# out_path = '../examples/threads/parsing/'
# write_page_data_to_csv(thread_data, headers, forum['fid'], out_path)

#### WRITE FORUM DATA TO CSV

In [296]:
def write_forum_data_to_csv(unpacked_data, csv_path, fid, data_types):

    for data_group, data_type in zip(unpacked_data, data_types):

        csv_file_path = create_csv_forum_file_path(csv_path, fid, data_type)
        
        df = pd.DataFrame(data_group)
        df.to_csv(csv_file_path, index=False, encoding='utf-8')

# test_data = setup_test_data()

# unpacked = test_data['unpacked']
# csv_path = test_data['csv_path']
# fid = test_data['fid']
# data_types = test_data['data_types']

# write_forum_data_to_csv(unpacked, csv_path, fid, data_types)
# # fid_39_posts.csv (81 lines), fid_39_users.csv (81 lines), fid_39_thanks.csv (0 lines)

#### REPORT TOTALS

In [297]:
# type(test_data['unpacked']) # List [[posts, users, thnaks]]
# type(test_data['unpacked'][0]) # List [[{<post>},{<post>}], [{<user>},{<user>}], [{<thanks>},{<thanks>}]}
# test_data['unpacked'][0][0] # Dict {<post>}

def report_totals(unpacked):
    # NOTE: THIS IS A POORLY EXTENSIBLE FUNCTION.
    # IT IS A QUICK UNEXPECTED REWRITE
        
    num_files = len(unpacked)
    header = 1
    
    total_posts = len(unpacked[0])
    total_users = len(unpacked[1])
    total_thanks = len(unpacked[2])
        
    csv_posts_lines = total_posts + header
    csv_users_lines = total_users + header
    csv_thanks_lines = total_thanks + header
    
    total_processed = total_posts + total_users + total_thanks
    total_csv_lines = csv_posts_lines + csv_users_lines + csv_thanks_lines
    
    print total_processed, 'PROCESSED --->', '\t', 'POSTS:', total_posts, '\t', 'USERS', total_users, '\t', 'THANKS:', total_thanks
    print total_csv_lines, 'CSV LINES --->', '\t', 'POSTS:', csv_posts_lines, '\t', 'USERS', csv_users_lines, '\t', 'THANKS:', csv_thanks_lines
    
    return collections.Counter({'posts': csv_posts_lines, 'users': csv_users_lines, 'thanks': csv_thanks_lines })

# report_unpacked_totals(test_data['unpacked'])

# # 160 PROCESSED ---> 	POSTS: 80 	USERS 80 	THANKS: 0
# # 163 CSV LINES ---> 	POSTS: 81 	USERS 81 	THANKS: 1
# # Counter({'posts': 81, 'thanks': 1, 'users': 81})

#### SPLIT LIST

In [298]:
# https://stackoverflow.com/a/43106405/6023530

def split_list(full_list, num_lists):
    return [array.tolist() for array in np.array_split(full_list, num_lists)]
      
# first_names = ['Steve', 'Jane', 'Sara', 'Mary','Jack','Bob', 'Bily', 'Boni', 'Chris','Sori', 'Will', 'Won','Li']
# split_list(first_names, 4)

#### PARSE FILE PATH

In [299]:
def parse_file_path(file_path):
            
    file_info = {}

    # fid_1506_tid_97875_pg_4.html'
    file_name = file_path.split('/')[-1]
    
    # ['fid', '1506', 'tid', '97875', 'pg', '4']
    file_path_data = file_name.split('.')[0].split('_')

    file_info['name'] = file_name
    file_info['fid'] = file_path_data[1]
    file_info['tid'] = file_path_data[3]
    file_info['pg'] = file_path_data[5]

    return file_info

# fp = '../examples/threads/parsing/csv/pages/posts/fid_1506_tid_97875_pg_4.html'
# fn = 'fid_1506_tid_97875_pg_4.html'
# file_info = parse_file_path(fn)
# fid_1506_tid_97875_pg_4.html 1506 97875 4
# print file_info['name'], file_info['fid'], file_info['tid'], file_info['pg'] 

#### SETUP TEST DATA

In [300]:
# TODO: FIND TEST PAGES WITH THANK YOUS

def setup_test_data(fid=39, page_list=[]):
    
    if not page_list:
    
        fid_39_tid_527690_pg_1 = '../examples/threads/parsing/39/fid_39_tid_527690_pg_1.html'
        fid_39_tid_81190_pg_7 = '../examples/threads/parsing/39/fid_39_tid_81190_pg_7.html'
        fid_39_tid_81190_pg_8 = '../examples/threads/parsing/39/fid_39_tid_81190_pg_8.html'
        
        page_list = [fid_39_tid_81190_pg_7, fid_39_tid_81190_pg_8, fid_39_tid_527690_pg_1]
    
    test_data = {}

    test_data['fid'] = str(fid)
    test_data['csv_path'] = '../data/csv/forums/'
    
    test_data['data_types'] = ['posts', 'users', 'thanks']
        
    test_data['parsed_pages'] = [parse_threads(page) for page in page_list]
    test_data['unpacked'] = unpack_parsed_pages(test_data['parsed_pages'], test_data['data_types'])
    
    return test_data

# test_data = setup_test_data()

### PARSING

#### UNPACK PARSED PAGES

In [301]:
def unpack_parsed_pages(pages, data_types):
        
    unpacked = []
    
    for data_type in data_types:

        unpacked.append([data for page in pages if page for data in page[data_type]])
        
    return unpacked

# # empty_lists = [{'posts': [], 'users': [], 'thanks': [] }]
# empty_dict = [{}]
# unpacked = unpack_parsed_pages(empty_lists, test_data['data_types'])

# # # WHAT THIS ACCOMPLISHES
# # posts = [post for parsed_page in parsed_pages for post in parsed_page['posts']]
# # users = [user for parsed_page in parsed_pages for post in parsed_page['users']]
# # thanks = [thank_you for parsed_page in parsed_pages for post in parsed_page['thanks']]

In [131]:
# html_file_path = '../examples/threads/parsing/39/fid_39_tid_527690_pg_1.html'
# html_page = load_html_file(html_file_path)
# html_posts = get_html_posts(html_page)
# html_thank_yous = get_html_thank_yous(html_page)
# post_thank_you_links = [get_thanks_links(html_thank_you) for html_thank_you in html_thank_yous]
# posts = [get_post_info(html_file_path, html_post) for html_post in html_posts]

#### CREATE LOG MESSAGES

In [302]:
def create_log_messages(html_file_path):
    
    fid, tid, pg = parse_file_path(html_file_path)
    
    file_info = '    FORUM: ' + fid + '    THREAD: ' + tid + '    PAGE: ' + pg + '    FILE: ' + file_name
    
    begin_msg ='PARSING' + file_info
    end_msg = 'SUCCESS'
        
    info_msg = begin_msg + '\tRESULT: ' + str(number) + ' * 2 = ' + str(doubled)
    error_msg  = begin_msg + '\tERR: NotDoubled ' + str(number) + ' <= ' + str(trigger)
    critical_msg  = begin_msg + '\tERR: '
    
    return {'info': info_msg, 'error': error_msg, 'critical': critical_msg}

# file_name = create_dummy_file(2, 4, '36', 'html') # u'2_doubled_4_pw_36.html'
# log_msgs = create_log_messages(2, 4, 1, '36', file_name)
# print log_msgs['info'], '\n', log_msgs['error'], '\n', log_msgs['critical']

In [165]:
def log_report(file_info, thread_data={}, status=''):
    
#     print status
    
    fid = file_info['fid']
    tid = file_info['tid']
    pg = file_info['pg']
    name = file_info['name']
    
    
    parsing = ' FILE: ' + name + '    FORUM: ' + fid + '    THREAD: ' + tid + '    PAGE: ' + pg
#     print parsing
        
    if thread_data:
        
        posts = len(thread_data['posts'])
        users = len(thread_data['users'])
        thanks = len(thread_data['thanks'])
        
        counts = ' FILE: ' + str(name) + '    POSTS: ' + str(posts) + '    USERS: ' + str(users) + '    THANKS: ' + str(thanks) 

        if status == 'parsing': # JUST STARTING
            report = status.upper() + ':' + parsing
        elif all([posts, users, thanks]): # IF ALL THREE HAVE CONTENT
            status = 'success:'
            report = status.upper() + ' PARSED' + counts
        elif all([posts, users]) and not thanks: # IF POSTS AND USERS HAVE CONTENT BUT NO THANKS MOST LIKELY NOT A PROBLEM
            status = 'warning:'
            report = status.upper() + ' PARSED' + counts
        elif any([posts, users]): # IF POSTS OR USERS IS EMPTY, HIGHLY LIKELY THERE IS A PROBLEM
            status = 'caution:'
            report = status.upper() + ' PARSED' + counts
        elif status != 'parsing' and not all([posts, users, thanks]):
            status = 'failure:'
            report = status.upper() + ' PARSED' + counts
        else:
            report = 'REPORTING ERROR'
    else:
        report = 'FAILURE: NO THREAD DATA'

#     print report

    logger.info(report)

#### PARSE THREADS

In [303]:
def parse_threads(html_file_path):
        
    thread_data = {'posts': [], 'users': [], 'thanks': []}
        
    try:
        print 'trying', html_file_path
        out_path = '../examples/threads/parsing/csv/'

        html_page = load_html_file(html_file_path)
        html_posts = get_html_posts(html_page)
        html_thank_yous = get_html_thank_yous(html_page)
        post_thank_you_links = [get_thanks_links(html_thank_you) for html_thank_you in html_thank_yous]
        
        
        posts = [get_post_info(html_file_path, html_post) for html_post in html_posts]
        
        users = [get_user_info(html_post) for html_post in html_posts]
        
        thank_yous = [get_thank_you_info(html_thank_you, thank_you_link) 
                                    for html_thank_you, thank_you_links in zip(html_thank_yous, post_thank_you_links) 
                                    for thank_you_link in thank_you_links]
                        
        # TO-DO: CHANGE WRITE_PAGE_DATA_TO_CSV TO ACCEPT DICT AS BELOW
        thread_data = [posts, users, thank_yous]
        
        # write_page_data_to_csv(thread_data, html_file_path, out_path)
        
#         fid, tid, pg = parse_file_path(html_file_path)
        
        thread_data = {'posts': posts, 'users': users, 'thanks': thank_yous}
        
#         return thread_data
        
    except Exception as err:
        print 'excepting', html_file_path, error_path
        shutil.copy2(html_file_path, error_path)
        file_name = html_file_path.split('/')[-1]
        logger.critical('CRITICAL ERROR PARSING FILE: ' + file_info['file_name'] + '    ERROR:'  + err.__class__.__name__ + ' ' + str(err))
        
        print file_name + ' - ' + err.__class__.__name__ + ': ' + str(err)


    finally:
        print 'returning thread_data', html_file_path
        return thread_data
        
# TODO: RIGHT NOW THIS IS DEPENDANT ON THE HEADERS AND THREAD DATA BEING IN THE SAME ORDER.  SHOULD PROBABLY MAKE BOTH INTO DICTIONARIES
# SO THAT A KEY CAN BE USED TO MATCH DATA TYPE TO DATA HEADER.

### PARSE DOCUMENTS

#### SETUP VARIABLES

In [ ]:
crld = False
mptest = False
stream = False
reset = True

# WHICH SERVER/ACTION

if crld:
    log_path = '../examples/logs/'
    error_path = '../examples/errors/'
    csv_path = '../data/csv/forums/'
    forums_dir = '../examples/threads/parsing/'
    processors = 1
elif mptest:
    log_path = 'logs/'
    error_path = 'errors/'
    csv_path = 'data/csv/forums/'
    forums_dir = 'tests/e3/'
    processors = 10
else:
    log_path = '../data/forums/logs/'
    error_path = '../data/forums/errors/'
    csv_path = '../data/forums/csv/'
    forums_dir = '../data/forums/'
    processors = 32

In [ ]:
# TODO: Figure out how to procure the UTC for 'Today' and 'Yesterday'

crld = True
mptest = True
stream = True
debug = True

# WHICH SERVER AM I ON

if crld:
    log_path = '../examples/logs/'
    error_path = '../examples/errors/'
    csv_path = '../data/csv/forums/'
    forums_dir = '../examples/threads/parsing/'
    processors = 1
elif mptest:
    log_path = 'logs/'
    error_path = 'errors/'
    csv_path = 'data/csv/forums/'
    forums_dir = '../threads/parsing'
    processors = 10

# STREAM LOGS TO STDOUT OR USE PROGRESS BAR

if stream:
    pbar = False
else:
    pbar = True

# RESET FOLDER CONDITIONS

if reset:
    # RESET LOGS
    shutil.rmtree(log_path)
    Path(log_path).mkdir()

    # RESET ERRORS
    shutil.rmtree(error_path)
    Path(error_path).mkdir()

parsed_forums = []
totals = collections.Counter()

# forums_dir = '../data/forums/'
skip = ['logs', 'csv', 'errors']
data_types = ['posts', 'users', 'thanks']

forum_paths = get_forum_directories(forums_dir, skip)

pages_by_forum = {forum_path.split('/')[-1] : [str(html_file) for html_file
                                               in Path(forum_path).glob('*.html')]
                                               for forum_path in forum_paths}

#### RUN PARSER

In [ ]:
for forum, html_file_paths in pages_by_forum.items():
    
    print 'FORUM:', forum, '\n'
    
    print len(html_file_paths), 'HTML FILES WILL NOW BE PROCESSED BY', processors, 'PROCESSOR(S)', '\n' # SHOULD BE 1679
    
    logger = create_multiprocessing_logger(log_path, 'fid_' + str(forum) + '.log', stream=stream)
    multiprocessing_logging.install_mp_handler(logger)
    
    parsed_pages = parmap.map(parse_threads, html_file_paths, pm_processes=processors, pm_bar=pbar)
    
    unpacked = unpack_parsed_pages(parsed_pages, data_types)
    
    write_forum_data_to_csv(unpacked, csv_path, forum, data_types)
            
    totals += report_totals(unpacked)
    
    print ''
    
print 'TOTAL LENGTH OF CSV\'s:', '\t', 'POSTS:', totals['posts'], '\t', 'USERS', totals['users'], '\t', 'THANKS:', totals['thanks']
# parsed_forums += [{'forum': forum, 'posts': posts, 'users': users, 'thanks': thanks}]

## SCRATCH

In [164]:
# finfo = {'fid': '39', 'tid': '81190', 'pg': '8', 'name': 'fid_39_tid_81190_pg_8.html'}

# parsing = {'posts': [], 'users': [], 'thanks': []} # PARSING

# all_data = {'posts': [0,1,2], 'users': [3,4,5,6,7], 'thanks': [3]} # SUCCESS
# no_thx = {'posts': [0,1,2], 'users': [3,4,5,6,7], 'thanks': []} # WARNING

# no_posts = {'posts': [], 'users': [3,4,5], 'thanks': [8,9]} # CAUTION
# no_users = {'posts': [0,1,2], 'users': [], 'thanks': [8,9]} # CAUTION
# no_posts_users = {'posts': [], 'users': [], 'thanks': [8,9]} # CAUTION

# no_data = {'posts': [], 'users': [], 'thanks': []} # FAILURE

# empty_data = {} # FAILURE NO DATA

# test_data = [all_data, no_thx, no_posts, no_users, no_posts_users, no_data, empty_data]

# log_report(finfo, thread_data=no_data, status='parsing')
# options = [log_report(finfo, thread_data=test_datum, status='') for test_datum in test_data]

# # PARSING: FILE: fid_39_tid_81190_pg_8.html    FORUM: 39    THREAD: 81190    PAGE: 8
# # SUCCESS: PARSED FILE: fid_39_tid_81190_pg_8.html    POSTS: 3    USERS: 5    THANKS: 1
# # WARNING: PARSED FILE: fid_39_tid_81190_pg_8.html    POSTS: 3    USERS: 5    THANKS: 0
# # CAUTION: PARSED FILE: fid_39_tid_81190_pg_8.html    POSTS: 0    USERS: 3    THANKS: 2
# # CAUTION: PARSED FILE: fid_39_tid_81190_pg_8.html    POSTS: 3    USERS: 0    THANKS: 2
# # FAILURE: PARSED FILE: fid_39_tid_81190_pg_8.html    POSTS: 0    USERS: 0    THANKS: 2
# # FAILURE: PARSED FILE: fid_39_tid_81190_pg_8.html    POSTS: 0    USERS: 0    THANKS: 0
# # FAILURE: NO THREAD DATA

./72 : 10496 file(s)
./44 : 25957 file(s)